In [1]:
## get data
import os
import wrangle

## analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

**Formula**  

- for each states:
    - **Note:** proportion of ppl get sick who are in labor force (current unemployment rate, by age)

$$unquarantine\_ change\_ rate=\sum_{i=0}^{n} \sum_{j=0}^{m}{proportion\_of\_state\_sick_{i,j}}\times{proportion\_of\_inds\_in\_state_{i,j}}$$
                        - n: number of states 
                        - m: number of industries


- Depending on how much data we have, we could put _shock_ in the formula (financial crsis, hospital bed overflow) 
    
- quarantine_change_rate (come back to this)
    - we should not assume quarantine rate for all the states, might need diff quarantine states
    - Jenn: current quarantine measures for each states
    - WFH possibility will diff from industry
    - layoffs

- Health care cost 

## Data
- employment 2019
- covid 19 confirmed cases
- states populations :https://www.census.gov/quickfacts/NY


### employment

In [2]:
employment_ny = wrangle.get_ny_employment()

In [3]:
employment_ny.head()

,naics_code,industry,2019,2021,net_change,pct_change
1,601,"Total Self Employed, All Jobs",570590,577460,6870,0.0120402
2,11,"Agriculture, Forestry, Fishing and Hunting",31180,31870,690,0.0221296
8,21,Mining,4230,4550,320,0.0756501
12,22,Utilities,36700,36430,-270,-0.00735695
13,23,Construction,380940,390570,9630,0.0252796


In [15]:
employment_nj = wrangle.get_nj_employment()

In [16]:
employment_nj.head()

,industry,2016,2026,net_change,pct_change
0,Total Nonfarm,4073600,4402180,328580,0.778744
1,Total Private Sector,3792200,4134190,341990,0.867189
2,Goods Producing,397000,429740,32740,0.795589
3,Natural Resources and Mining,1300,1400,100,0.743833
4,Construction,153600,176260,22660,1.3856


### covid 19

In [4]:
covid19_us = wrangle.get_covid_data(date = "03-29-2020")

In [5]:
ny_confirmed = covid19_us.loc[covid19_us["Province_State"] == "New York", "Confirmed"].sum()

### population

In [6]:
population_2019 = wrangle.get_population_data()

In [7]:
population_2019.head()

,SUMLEV,REGION,DIVISION,STATE,NAME,POPESTIMATE2019,POPEST18PLUS2019,PCNT_POPEST18PLUS
0,10,0,0,0,United States,328239523,255200373,77.7
1,40,3,6,1,Alabama,4903185,3814879,77.8
2,40,4,9,2,Alaska,731545,551562,75.4
3,40,4,8,4,Arizona,7278717,5638481,77.5
4,40,3,7,5,Arkansas,3017804,2317649,76.8


In [8]:
ny_pop = population_2019.loc[population_2019["NAME"] == "New York","POPESTIMATE2019"].values[0]

In [9]:
ny_confirmed/ny_pop

0.0030661738485822723

### GDP

In [10]:
gdp_dir = os.path.join(os.getcwd(),"data/qgdpstate0120_2.pdf")

In [11]:
gdp_states = wrangle.get_gdp_by_states(data_dir=gdp_dir)

In [12]:
gdp_states_inds = wrangle.get_gdp_pct_by_states_by_inds(data_dir=gdp_dir)

In [13]:
gdp_states.head()

,areas,2018-1,2018-2,2018-3,2018-4,2019-1,2019-2,2019-3
0,United States,"20,163,159","20,510,177","20,749,752","20,897,804","21,098,827","21,340,267","21,542,540"
1,New England,"1,070,863","1,084,947","1,096,174","1,101,531","1,118,703","1,130,084","1,142,586"
2,Connecticut,"272,854","273,884","278,295","277,874","281,659","284,357","287,560"
3,Maine,"63,698","64,746","65,434","65,545","66,590","67,138","67,905"
4,Massachusetts,"558,472","568,220","573,506","577,754","586,347","592,588","599,092"


In [14]:
gdp_states_inds.head()

,areas,states_overall,agriculture_forestry_fishing_insuranceand_hunting,mining_quarrying_and_oil_and_gas_extraction,utilities,construction,retail_trade,transportation_and_warehousing,durable_goods_manufacturing,nondurable_goods_manufacturing,...,management_of_companies_and_enterprises,administrative_and_support_and_waste_management_and_remediation_services,arts_entertainment_and_recreation,accomodation_and_food_services,real_estate_and_rental_and_leasing,professional_scientific_and_technical_services,educational_services,health_care_and_social_assistance,other_services_except_government_and_government_enterprises_,government_and_government_enterprises
0,United States,2.1,0.06,0.11,-0.26,-0.09,0.43,-0.03,0.01,0.46,...,0.16,0.10,0.02,0.17,0.11,0.42,0.08,0.19,0.05,0.01
1,New England,2.1,0.01,0.01,-0.24,-0.09,0.39,-0.02,0.02,0.33,...,0.20,0.12,0.04,0.17,0.09,0.56,0.23,0.21,0.05,-0.01
2,Connecticut,2.1,0.00,0.00,-0.27,-0.15,0.39,-0.01,0.02,0.71,...,0.19,0.11,0.04,0.11,0.03,0.37,0.27,0.17,0.07,-0.04
3,Maine,2.1,0.01,0.00,-0.22,0.04,0.72,0.01,-0.02,0.29,...,0.27,0.15,0.05,0.33,0.07,0.26,0.06,0.15,0.05,-0.10
4,Massachusetts,2.2,0.02,0.01,-0.21,-0.11,0.31,-0.03,0.04,0.21,...,0.20,0.10,0.03,0.16,0.13,0.72,0.24,0.22,0.05,0.00


In [10]:
def get_unquarantine_change_rate():
    
    return 0
    